In [1]:
from google.colab import drive

In [4]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
cd drive/MyDrive/Kaggle/Google\ Brain\ Ventilator\ Pressure/ventilator-pressure

/content/drive/MyDrive/Kaggle/Google Brain Ventilator Pressure/ventilator-pressure


In [6]:
import notebook_module
notebook_module.execute_hook()

/usr/local/lib/python3.7/dist-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


In [7]:
import pandas as pd
import data_preparation

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split

import numpy as np

importing Jupyter notebook from data_preparation.ipynb


In [8]:
df = pd.read_csv('data/train.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
df = data_preparation.xgboost_data_preparation(df)

In [10]:
df.head()

,breath_id,time_step,u_in,u_out,pressure,R_5,R_20,R_50,C_10,C_20,C_50
id,,,,,,,,,,,
1,1,0.000000,0.083334,0,5.837492,0,1,0,0,0,1
2,1,0.033652,18.383041,0,5.907794,0,1,0,0,0,1
3,1,0.067514,22.509278,0,7.876254,0,1,0,0,0,1
4,1,0.101542,22.808822,0,11.742872,0,1,0,0,0,1
5,1,0.135756,25.355850,0,12.234987,0,1,0,0,0,1


## Reshape Data in to Row Vectors

In [16]:
df_reshape_values = df[['time_step', 'u_in', 'u_out']]
reshaped_arr = df_reshape_values.values.reshape(len(df_reshape_values)//80, 80*3)

In [17]:
lung_data_arr = df.loc[df['time_step'] == 0, ['R_5', 'R_20', 'R_50', 'C_10', 'C_20', 'C_50']].values

In [18]:
columns = [[f'time_step_{x}', f'u_in_{x}', f'u_out_{x}'] for x in range(1,81)]
columns = np.array(columns).reshape(80*3)
columns = np.append(columns, ['R_5', 'R_20', 'R_50', 'C_10', 'C_20', 'C_50'])

data = np.column_stack((reshaped_arr, lung_data_arr))

df_reshaped = pd.DataFrame(data, columns=columns, index=df['breath_id'].unique())
df_reshaped.index.name = 'breath_id'

## Pick First Pressure to Predict

In [29]:
target_1 = df.groupby('breath_id')['pressure'].nth(0)

In [30]:
df_train = df_reshaped.copy()
df_train['pressure'] = target_1.values

## Train the Model

In [ ]:
X, X_val, y, y_val = train_test_split(df_train.drop('pressure', axis=1), df_train['pressure'], test_size=0.1)
model = XGBRegressor(tree_method='gpu_hist', objective='reg:squarederror', max_depth=10, n_estimators=100000, learning_rate=0.02)
model.fit(X, y, eval_set=[(X_val, y_val)], eval_metric='mae', verbose=1, early_stopping_rounds=200)

## Predictions on the Dataset

In [34]:
predictions = model.predict(df_train.drop('pressure', axis=1))

In [35]:
predictions

array([5.8392205, 6.2039084, 5.084789 , ..., 6.2448883, 6.8764534,
       3.8974285], dtype=float32)

## Add Predictions to the Dataset

In [43]:
df_train['pressure_step_1'] = predictions

## Set Pressure at Step 2 as Target

In [44]:
target_1 = df.groupby('breath_id')['pressure'].nth(1)

In [45]:
df_train['pressure'] = target_1.values

## Train the Model for Step 2 without previous predictions

In [46]:
X, X_val, y, y_val = train_test_split(df_train.drop(['pressure', 'pressure_step_1'], axis=1), df_train['pressure'], test_size=0.1)
model = XGBRegressor(tree_method='gpu_hist', objective='reg:squarederror', max_depth=10, n_estimators=100000, learning_rate=0.02)
model.fit(X, y, eval_set=[(X_val, y_val)], eval_metric='mae', verbose=1, early_stopping_rounds=200)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
[2060]	validation_0-mae:0.162692
[2061]	validation_0-mae:0.162692
[2062]	validation_0-mae:0.162689
[2063]	validation_0-mae:0.162689
[2064]	validation_0-mae:0.162689
[2065]	validation_0-mae:0.162687
[2066]	validation_0-mae:0.162688
[2067]	validation_0-mae:0.162688
[2068]	validation_0-mae:0.162687
[2069]	validation_0-mae:0.162687
[2070]	validation_0-mae:0.162685
[2071]	validation_0-mae:0.162687
[2072]	validation_0-mae:0.162687
[2073]	validation_0-mae:0.162688
[2074]	validation_0-mae:0.162693
[2075]	validation_0-mae:0.162695
[2076]	validation_0-mae:0.162698
[2077]	validation_0-mae:0.162703
[2078]	validation_0-mae:0.162703
[2079]	validation_0-mae:0.162704
[2080]	validation_0-mae:0.162703
[2081]	validation_0-mae:0.162703
[2082]	validation_0-mae:0.162699
[2083]	validation_0-mae:0.162699
[2084]	validation_0-mae:0.162698
[2085]	validation_0-mae:0.162699
[2086]	validation_0-mae:0.162697
[2087]	validation_0-mae:0.162694
[2088]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None,
             n_estimators=100000, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, tree_method='gpu_hist', verbosity=1)

## Train the Model for Step 2 with previous predictions

In [47]:
X, X_val, y, y_val = train_test_split(df_train.drop(['pressure'], axis=1), df_train['pressure'], test_size=0.1)
model = XGBRegressor(tree_method='gpu_hist', objective='reg:squarederror', max_depth=10, n_estimators=100000, learning_rate=0.02)
model.fit(X, y, eval_set=[(X_val, y_val)], eval_metric='mae', verbose=1, early_stopping_rounds=200)

[0]	validation_0-mae:5.94561
Will train until validation_0-mae hasn't improved in 200 rounds.
[1]	validation_0-mae:5.82679
[2]	validation_0-mae:5.71035
[3]	validation_0-mae:5.59617
[4]	validation_0-mae:5.48433
[5]	validation_0-mae:5.37472
[6]	validation_0-mae:5.26734
[7]	validation_0-mae:5.16208
[8]	validation_0-mae:5.05888
[9]	validation_0-mae:4.95776
[10]	validation_0-mae:4.85864
[11]	validation_0-mae:4.7615
[12]	validation_0-mae:4.66645
[13]	validation_0-mae:4.57321
[14]	validation_0-mae:4.48179
[15]	validation_0-mae:4.39229
[16]	validation_0-mae:4.30447
[17]	validation_0-mae:4.21843
[18]	validation_0-mae:4.13418
[19]	validation_0-mae:4.05162
[20]	validation_0-mae:3.97063
[21]	validation_0-mae:3.89128
[22]	validation_0-mae:3.81349
[23]	validation_0-mae:3.73731
[24]	validation_0-mae:3.66266
[25]	validation_0-mae:3.58948
[26]	validation_0-mae:3.51772
[27]	validation_0-mae:3.44747
[28]	validation_0-mae:3.37857
[29]	validation_0-mae:3.3111
[30]	validation_0-mae:3.24493
[31]	validation_0

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None,
             n_estimators=100000, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, tree_method='gpu_hist', verbosity=1)

Conclusion the Model with previous predictions is way superior